In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pickle
import os
import glob
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
import torch.nn as nn

from torch.utils.data import DataLoader
from torch.utils.data.dataset import TensorDataset

In [3]:
from fastai.vision import *
from fastai.metrics import error_rate
from fastai import *

## Dataset

In [4]:
def generate_data(size):
    x = np.random.uniform(size=(size, 1))
    y = x * 2.0
    return torch.FloatTensor(x), torch.FloatTensor(y)

train_x, train_y = generate_data(1000)
val_x, val_y = generate_data(100)

## Model

In [5]:
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.linear1 = nn.Linear(1, 5)
        self.linear2 = nn.Linear(5, 1)

    def forward(self, x):
        x = self.linear1(x)
        x = self.linear2(x)

        return x

## Training

In [6]:
train_ds = TensorDataset(train_x, train_y)
val_ds = TensorDataset(val_x, val_y)

train_dl = DataLoader(train_ds, batch_size=8)
val_dl = DataLoader(val_ds, batch_size=8)

data_bunch = DataBunch(train_dl, val_dl)

In [7]:
model = SimpleModel()
learn = Learner(data_bunch, model, loss_func=F.mse_loss)
learn.fit_one_cycle(10)

epoch,train_loss,valid_loss,time
0,2.947107,2.804191,00:00
1,1.444926,0.712821,00:00
2,0.233015,0.068923,00:00
3,0.051762,0.020974,00:00
4,0.010390,0.002592,00:00
5,0.001259,0.000093,00:00
6,0.000111,0.000002,00:00
7,0.000009,0.000000,00:00
8,0.000001,0.000000,00:00
9,0.000000,0.000000,00:00


In [8]:
model = SimpleModel()
learn = Learner(data_bunch, model, loss_func=F.mse_loss, layer_groups=[model.linear1, model.linear2])
learn.fit_one_cycle(5)

epoch,train_loss,valid_loss,time
0,2.451575,2.638573,00:00
1,2.451574,2.638573,00:00
2,2.451574,2.638573,00:00
3,2.451574,2.638573,00:00
4,2.451575,2.638573,00:00


In [9]:
model = SimpleModel()
layer_groups=[nn.Sequential(model.linear1), nn.Sequential(model.linear2)]
learn = Learner(data_bunch, model, loss_func=F.mse_loss, layer_groups=layer_groups)
learn.fit_one_cycle(5)

epoch,train_loss,valid_loss,time
0,0.435890,0.264410,00:00
1,0.155028,0.104372,00:00
2,0.087640,0.065839,00:00
3,0.058719,0.048342,00:00
4,0.049846,0.045488,00:00
